In [3]:
from tgate_counter import count_tgate
from prepare_circuit import uniform_layered_m_body, dist_select_m_body
from statistics import mean, pstdev
from multiprocessing import cpu_count
# 標準のmultiprocessingの代わりにmpを使わないとトップスコープの名前が解決されない
import multiprocess as mp
import pyzx as zx

def print_result(result_dict):
    size = result_dict["size"]
    count = result_dict["count"]
    before = result_dict["before"]
    mean = result_dict["mean"]
    stdev = result_dict["stdev"]
    parallel = result_dict["parallel"]
    print("=====")
    print("stat: size ->", 2**size, ", count ->", count, ", parallel ->", parallel)
    print("before t-depth:", before)
    print("after t-depth:", mean, "+/-", stdev)
    imp = (1 - mean/before)*100
    print("improvement: ", imp, "%")

def reduce_qasm(qasm_str: str) -> int:
    qasm_circuit = zx.Circuit.from_qasm(qasm_str)
    g = qasm_circuit.to_graph()
    zx.draw(g) # draw circuit
    zx.full_reduce(g)
    g.normalize()
    c_opt = zx.extract_circuit(g.copy())
    after_qasm = c_opt.to_qasm()
    return count_tgate(after_qasm)[1]

def experiment_m_body(size: int, interaction: int, parallel: int, count: int):
    # prepare qasms before optimization
    before_qasms: list[str] = []
    for _ in range(count):
        if parallel == 0:
            qasm = uniform_layered_m_body(size, interaction)
        else:
            qasm = dist_select_m_body(size, interaction, parallel)
        before_qasms.append(qasm)
    before = mean([count_tgate(qasm)[1] for qasm in before_qasms])
    # concurrent optimization
    with mp.Pool(cpu_count()) as pool:
        afters: list[int] = pool.map(reduce_qasm, before_qasms)
    avg = mean(afters)
    stdev = pstdev(afters)
    return {
        "size": size,
        "count": count,
        "before": before,
        "mean": avg,
        "stdev": stdev,
        "parallel": parallel
    }

In [4]:
print_result(experiment_m_body(4, 1, 0, 10))
print_result(experiment_m_body(4, 1, 1, 10))
print_result(experiment_m_body(4, 1, 2, 10))
print_result(experiment_m_body(4, 1, 3, 10))

=====
stat: size -> 16 , count -> 10 , parallel -> 0
before t-depth: 92
after t-depth: 98 +/- 4.427188724235731
improvement:  -6.521739130434789 %
=====
stat: size -> 16 , count -> 10 , parallel -> 1
before t-depth: 92
after t-depth: 91.2 +/- 11.565465835840769
improvement:  0.8695652173912993 %
=====
stat: size -> 16 , count -> 10 , parallel -> 2
before t-depth: 38
after t-depth: 70.2 +/- 8.55336191213724
improvement:  -84.73684210526315 %
=====
stat: size -> 16 , count -> 10 , parallel -> 3
before t-depth: 44
after t-depth: 88.6 +/- 11.29778739399888
improvement:  -101.36363636363637 %


In [5]:
print_result(experiment_m_body(4, 2, 0, 10))
print_result(experiment_m_body(4, 2, 1, 10))
print_result(experiment_m_body(4, 2, 2, 10))

=====
stat: size -> 16 , count -> 10 , parallel -> 0
before t-depth: 92
after t-depth: 97.7 +/- 3.5791060336346563
improvement:  -6.195652173913047 %
=====
stat: size -> 16 , count -> 10 , parallel -> 1
before t-depth: 92
after t-depth: 92.1 +/- 11.335342959081565
improvement:  -0.10869565217390686 %
=====
stat: size -> 16 , count -> 10 , parallel -> 2
before t-depth: 38.8
after t-depth: 68 +/- 7.483314773547883
improvement:  -75.2577319587629 %


In [6]:
print_result(experiment_m_body(4, 3, 0, 10))
print_result(experiment_m_body(4, 3, 1, 10))
print_result(experiment_m_body(4, 3, 2, 10))

=====
stat: size -> 16 , count -> 10 , parallel -> 0
before t-depth: 92
after t-depth: 99.3 +/- 3.606937759374287
improvement:  -7.934782608695645 %
=====
stat: size -> 16 , count -> 10 , parallel -> 1
before t-depth: 92
after t-depth: 92.3 +/- 11.75627492022877
improvement:  -0.3260869565217428 %
=====
stat: size -> 16 , count -> 10 , parallel -> 2
before t-depth: 38.8
after t-depth: 72.4 +/- 7.418894796396563
improvement:  -86.59793814432992 %


In [7]:
print_result(experiment_m_body(5, 1, 0, 10))
print_result(experiment_m_body(5, 1, 1, 10))
print_result(experiment_m_body(5, 1, 2, 10))

=====
stat: size -> 32 , count -> 10 , parallel -> 0
before t-depth: 188
after t-depth: 201.8 +/- 5.418486873657627
improvement:  -7.340425531914896 %
=====
stat: size -> 32 , count -> 10 , parallel -> 1
before t-depth: 188
after t-depth: 150.3 +/- 27.19577173017894
improvement:  20.053191489361698 %
=====
stat: size -> 32 , count -> 10 , parallel -> 2
before t-depth: 62
after t-depth: 127.3 +/- 18.708554193202637
improvement:  -105.3225806451613 %


In [9]:
print_result(experiment_m_body(5, 2, 0, 10))
print_result(experiment_m_body(5, 2, 1, 10))
print_result(experiment_m_body(5, 2, 2, 10))

=====
stat: size -> 32 , count -> 10 , parallel -> 0
before t-depth: 188
after t-depth: 201.6 +/- 7.045565981523415
improvement:  -7.234042553191489 %
=====
stat: size -> 32 , count -> 10 , parallel -> 1
before t-depth: 188
after t-depth: 185.2 +/- 23.886397802933786
improvement:  1.4893617021276673 %
=====
stat: size -> 32 , count -> 10 , parallel -> 2
before t-depth: 71.6
after t-depth: 126 +/- 21.945386758952324
improvement:  -75.97765363128492 %


In [10]:
print_result(experiment_m_body(5, 3, 0, 10))
print_result(experiment_m_body(5, 3, 1, 10))
print_result(experiment_m_body(5, 3, 2, 10))

=====
stat: size -> 32 , count -> 10 , parallel -> 0
before t-depth: 188
after t-depth: 200.6 +/- 7.1442284397967
improvement:  -6.702127659574475 %
=====
stat: size -> 32 , count -> 10 , parallel -> 1
before t-depth: 188
after t-depth: 176.1 +/- 21.130309983528402
improvement:  6.329787234042561 %
=====
stat: size -> 32 , count -> 10 , parallel -> 2
before t-depth: 78.4
after t-depth: 117.6 +/- 17.030560765870277
improvement:  -49.99999999999998 %


In [11]:
print_result(experiment_m_body(6, 1, 0, 10))
print_result(experiment_m_body(6, 1, 1, 10))
print_result(experiment_m_body(6, 1, 2, 10))

=====
stat: size -> 64 , count -> 10 , parallel -> 0
before t-depth: 380
after t-depth: 402.9 +/- 6.80367547726962
improvement:  -6.02631578947368 %
=====
stat: size -> 64 , count -> 10 , parallel -> 1
before t-depth: 380
after t-depth: 297.1 +/- 58.834428696129955
improvement:  21.815789473684198 %
=====
stat: size -> 64 , count -> 10 , parallel -> 2
before t-depth: 110
after t-depth: 177.8 +/- 20.033971149025845
improvement:  -61.63636363636365 %


In [12]:
print_result(experiment_m_body(6, 2, 0, 10))
print_result(experiment_m_body(6, 2, 1, 10))
print_result(experiment_m_body(6, 2, 2, 10))
print_result(experiment_m_body(6, 3, 0, 10))
print_result(experiment_m_body(6, 3, 1, 10))
print_result(experiment_m_body(6, 3, 2, 10))

=====
stat: size -> 64 , count -> 10 , parallel -> 0
before t-depth: 380
after t-depth: 400.1 +/- 8.018104514160438
improvement:  -5.289473684210533 %
=====
stat: size -> 64 , count -> 10 , parallel -> 1
before t-depth: 380
after t-depth: 297.6 +/- 40.22486793017474
improvement:  21.684210526315784 %
=====
stat: size -> 64 , count -> 10 , parallel -> 2
before t-depth: 136.8
after t-depth: 201.4 +/- 25.160286166894046
improvement:  -47.22222222222221 %
=====
stat: size -> 64 , count -> 10 , parallel -> 0
before t-depth: 380
after t-depth: 404.3 +/- 8.12465383877984
improvement:  -6.394736842105275 %
=====
stat: size -> 64 , count -> 10 , parallel -> 1
before t-depth: 380
after t-depth: 291.8 +/- 42.13976744121875
improvement:  23.21052631578947 %
=====
stat: size -> 64 , count -> 10 , parallel -> 2
before t-depth: 159.2
after t-depth: 196.7 +/- 21.29812198293549
improvement:  -23.55527638190955 %
